In [1]:
from rich.pretty import Pretty
from sigpyproc.readers import FilReader
from your.formats.filwriter import make_sigproc_object
from your import Your, Writer
import os
import numpy as np
import tempfile
from urllib.request import urlretrieve

### Header form original file (the original header of the observation, that specifies the central frequency, etc etc)

Header read with readfile from PRESTO

1: From the SIGPROC filterbank file 'ds8192_J1810-197_A2_20220929_180235.fil':
Telescope = IAR1

Source Name = J1810-197_A2

Backend = Ettus-B205

Obs Date String = 2022-09-29T21:02:38

MJD start time = 59851.87682870370190

RA J2000 = 18:09:51.0800

RA J2000 (deg) = 272.462833333333 

Dec J2000 = -19:43:51.9300

Dec J2000 (deg) = -19.7310916666667

Tracking? = True

Azimuth (deg) = 0

Zenith Ang (deg) = 0

Number of polns = 2 (summed)

Sample time (us) = 146.285714285714

Central freq (MHz) = 1428.875 

Low channel (MHz) = 1401.75  

High channel (MHz) = 1456
Channel width (MHz) = 1.75             

Number of channels = 32
Total Bandwidth (MHz) = 56

Beam = 1 of 1

Beam FWHM (deg) = 1.000

Spectra per subint = 480

Spectra per file = 58195968

Time per subint (sec) = 0.0702171428571

Time per file (sec) = 8513.23874743

bits per sample = 32

bytes per spectra = 128
samples per spectra = 32

bytes per subint = 61440

samples per subint = 15360

zero offset = 0

Invert the band? = False

bytes in file header = 385



### Reading the paths

In [2]:
path_fil_original = 'su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/ds8192_J1810-197_A2_20220929_180235.fil'

In [3]:
path_your = 'su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/filfromfil1.fil'

### Reading the files and triying to invert them with sigpyproc

In [63]:
filoriginal = FilReader(path_fil_original)  #this is the original file
filyour = FilReader(path_your)   #This is one that I made with your, supposelly with the inverted band, but the fch1 channel is wrong, is at 1510 MHz

The original file has foff=1.75, and fch1=1456.0, and when PRESTO's waterfaller.py reads it, it thinks that fch1 is the first channel, and it should be the last one.

I understand that either I have to change fch1 to be fch1=1401.75, which is the original first channel, or changing foff=-1.75. 

But I don't seem to be able to change just one of those settings

In [38]:
Pretty(filoriginal.header)  #the original file



Header(
    filename='su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/ds8192_J1810-197_A2_20220929_180235
.fil',
    data_type='filterbank',
    nchans=32,
    foff=1.75,
    fch1=1456.0,
    nbits=32,
    tsamp=0.00014628571428571428,
    tstart=59851.8768287037,
    nsamples=58195968,
    nifs=1,
    coord=<SkyCoord (ICRS): (ra, dec) in deg
    (272.46283333, -19.73109167)>,
    azimuth=<Angle 0. deg>,
    zenith=<Angle 0. deg>,
    telescope='Fake',
    backend='FAKE',
    source='J1810-197_A2',
    frame='topocentric',
    ibeam=1,
    nbeams=1,
    dm=0,
    period=0,
    accel=0,
    signed=False,
    rawdatafile='ds8192_J1810-197_A2_20220929_180235.fil',
    hdrlens=[385],
    datalens=[7449083904],
    filenames=[
        'su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/ds8192_J1810-197_A2_20220929_180235.fil'
    ],
    nsamples_files=[58195968],
    tstart_files=[59851.8768287037]
)

In [39]:
Pretty(filyour.header) 
#This .fil file has the wrong first channel. Also the telescope parameter isn't right

Header(
    filename='su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/filfromfil1.fil',
    data_type='filterbank',
    nchans=32,
    foff=-1.75,
    fch1=1510.25,
    nbits=32,
    tsamp=0.00014628571428571428,
    tstart=59851.8768287037,
    nsamples=15360,
    nifs=1,
    coord=<SkyCoord (ICRS): (ra, dec) in deg
    (272.46283333, -19.73109167)>,
    azimuth=<Angle -1. deg>,
    zenith=<Angle -1. deg>,
    telescope='GBT',
    backend='FAKE',
    source='J1810-197_A2',
    frame='topocentric',
    ibeam=0,
    nbeams=1,
    dm=0,
    period=0,
    accel=0,
    signed=False,
    rawdatafile='./filfromfil1.fil',
    hdrlens=[403],
    datalens=[1966080],
    filenames=['su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/filfromfil1.fil'],
    nsamples_files=[15360],
    tstart_files=[59851.8768287037]
)

Triying invert_freq() from sigproc

In [26]:
filyour.invert_freq()

Output()

'filfromfil1_inverted.fil'

In [66]:
path_inverted_your = "filfromfil1_inverted.fil"

In [41]:
filoriginal.invert_freq()


Output()

In [69]:
path_inverted_original = "./ds8192_J1810-197_A2_20220929_180235_inverted.fil"

In [64]:
type(filoriginal.header)


sigpyproc.header.Header

In [67]:
Pretty(FilReader(path_inverted_your).header)
#here I get the same configuration from my original file, so it doesn't fix the problem

Header(
    filename='filfromfil1_inverted.fil',
    data_type='filterbank',
    nchans=32,
    foff=1.75,
    fch1=1456.0,
    nbits=32,
    tsamp=0.00014628571428571428,
    tstart=59851.8768287037,
    nsamples=15360,
    nifs=1,
    coord=<SkyCoord (ICRS): (ra, dec) in deg
    (272.46283333, -19.73109167)>,
    azimuth=<Angle -1. deg>,
    zenith=<Angle -1. deg>,
    telescope='GBT',
    backend='FAKE',
    source='J1810-197_A2',
    frame='topocentric',
    ibeam=0,
    nbeams=1,
    dm=0.0,
    period=0,
    accel=0,
    signed=0,
    rawdatafile='./filfromfil1.fil',
    hdrlens=[431],
    datalens=[1966080],
    filenames=['filfromfil1_inverted.fil'],
    nsamples_files=[15360],
    tstart_files=[59851.8768287037]
)

In [70]:
Pretty(FilReader(path_inverted_original).header)
#So if I invert the .fil file it seems that it changes the sing of 'foff' and the value of fch1, I would like to change just one 
#Here I have again the problem that the fch1 es 1510, which isn't an observed frequency, and the same result I got in the Your file

Header(
    filename='./ds8192_J1810-197_A2_20220929_180235_inverted.fil',
    data_type='filterbank',
    nchans=32,
    foff=-1.75,
    fch1=1510.25,
    nbits=32,
    tsamp=0.00014628571428571428,
    tstart=59851.8768287037,
    nsamples=58195968,
    nifs=1,
    coord=<SkyCoord (ICRS): (ra, dec) in deg
    (272.46283333, -19.73109167)>,
    azimuth=<Angle 0. deg>,
    zenith=<Angle 0. deg>,
    telescope='Fake',
    backend='FAKE',
    source='J1810-197_A2',
    frame='topocentric',
    ibeam=1,
    nbeams=1,
    dm=0.0,
    period=0,
    accel=0,
    signed=0,
    rawdatafile='ds8192_J1810-197_A2_20220929_180235.fil',
    hdrlens=[453],
    datalens=[7449083904],
    filenames=['./ds8192_J1810-197_A2_20220929_180235_inverted.fil'],
    nsamples_files=[58195968],
    tstart_files=[59851.8768287037]
)

### Triying to invert with your

Page --> https://thepetabyteproject.github.io/your/0.6.6/ipynb/Writer/#writing-filterbanks-from-scratch

In [59]:
#Let's see the original filterbank parameters in the your format:

Your_original_fil = Your(path_fil_original)
Your_original_fil.your_header

#Here we can already see that Your thinks that the center_freq is higher than the one it really is

Unified Header:
  "basename": "ds8192_J1810-197_A2_20220929_180235"
  "bw": 56.0
  "center_freq": 1483.125
  "dec_deg": -19.731091666666668
  "dtype": "float32"
  "fch1": 1456.0
  "filelist": [
    "su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/ds8192_J1810-197_A2_20220929_180235.fil"
  ]
  "filename": "su_2023/J1810-197_ettus/2022-09-29T21:01:02.422413_A2/filoriginal/ds8192_J1810-197_A2_20220929_180235.fil"
  "foff": 1.75
  "format": "fil"
  "frequency_decimation_factor": 1
  "gb": -0.15836762417033684
  "gl": -169.27316727335108
  "native_foff": 1.75
  "native_nbits": 32
  "native_nchans": 32
  "native_nspectra": 58195968.0
  "native_tsamp": 0.00014628571428571428
  "nbits": 32
  "nchans": 32
  "npol": 1
  "nspectra": 58195968
  "poln_order": "I"
  "ra_deg": 272.4628333333333
  "source_name": "J1810-197_A2"
  "time_decimation_factor": 1
  "tsamp": 0.00014628571428571428
  "tstart": 59851.8768287037
  "tstart_utc": "2022-09-29T21:02:38.000"

#### Making a sigproc_object first so I can specify the first channel and the foff with your

So with this, I get the right center frequency, but I don't know how to define the other header parameters correctly, 
so it raises some warnings and sigpyproc can't read the final .fil

In [24]:
sigproc_object = make_sigproc_object(
    rawdatafile  = path_fil_original,
    source_name = "J1810-197_A2",
    nchans  = 32, 
    foff=-1.75, #MHz
    fch1 = 1456.0, # MHz
    tsamp = 0.00014628571428571428, # seconds
    tstart = 59851.8768287037, #MJD
    src_raj=272.46283333, # HHMMSS.SS
    src_dej=-19.73109167, # DDMMSS.SS
    machine_id=0,
    nbeams=1,
    ibeam=1,
    nifs=1,
    barycentric=0,
    pulsarcentric=0,
)

#So my problem now is that this way it has the right range of frequencies, but almost all of the other header parameters aren't right, even if
#I pass the right values

In [25]:
sigproc_object.write_header("foo.fil")
Your("foo.fil").your_header


Unified Header:
  "basename": "foo"
  "bw": -56.0
  "center_freq": 1428.875
  "dec_deg": null
  "dtype": "uint8"
  "fch1": 1456.0
  "filelist": [
    "foo.fil"
  ]
  "filename": "foo.fil"
  "foff": -1.75
  "format": "fil"
  "frequency_decimation_factor": 1
  "gb": null
  "gl": null
  "native_foff": -1.75
  "native_nbits": 8
  "native_nchans": 32
  "native_nspectra": 7.0
  "native_tsamp": 0.00014628571428571428
  "nbits": 8
  "nchans": 32
  "npol": 1
  "nspectra": 7
  "poln_order": "I"
  "ra_deg": null
  "source_name": "TEMP"
  "time_decimation_factor": 1
  "tsamp": 0.00014628571428571428
  "tstart": 59851.8768287037
  "tstart_utc": "2022-09-29T21:02:38.000"

##### Here I'm triying to read it with Sigpyproc and it fails, so there's some problem with the way the header was writen


In [62]:
FilReader("foo.fil").header

ValueError: Cannot parse first argument data "0 2 72.46283333000002" for attribute ra

#### After not getting the right header, I tried with your.Writer, and I had the same problem that with sigpyproc, so it wasn't a solution

I can specify if the highest_frequency is the first, and the channels, which one is fist or not. But when triying this, I coulnd't get the right frequency interval that we observed

In [39]:
Writer?

Init signature:
Writer(
    your_object,
    nstart=0,
    nsamp=None,
    c_min=None,
    c_max=None,
    npoln=1,
    highest_frequency_first=False,
    outdir=None,
    outname=None,
    flag_rfi=False,
    progress=True,
    spectral_kurtosis_sigma=4,
    savgol_frequency_window=15,
    savgol_sigma=4,
    gulp=None,
    zero_dm_subt=False,
    time_decimation_factor=1,
    frequency_decimation_factor=1,
    replacement_policy='mean',
)
Docstring:     
The unified writer class.

Args:

    your_object: Your object
    nstart (int): Start sample to read from
    nsamp (int): Number of samples to write
    c_min (int): Starting channel index (default: 0)
    c_max (int): End channel index (default: total number of frequencies)
    npoln (int): Number of output polarisations (default: 1)
    highest_frequency_first (bool): The output should have highest frequency first (default: False)
    outdir (str): Output directory for file
    outname (str): Name of the file to write to (without

In [51]:
writer_object = Writer(
    ...: Your(path_fil_original),
    ...: nstart=0,
    ...: nsamp=15360,
    ...: c_min=31,
    ...: c_max=0,
    ...: highest_frequency_first=True,
    ...: outdir ="./",
    ...: outname = "filfromfil2",
    ...: )


In [52]:
writer_object.to_fil()

Output()

In [54]:
Your("./filfromfil2.fil").your_header
#Let's remember that the central frequency is 1428.875 MHz

Unified Header:
  "basename": "filfromfil2"
  "bw": -1.75
  "center_freq": 1510.25
  "dec_deg": -19.731091666666668
  "dtype": "float32"
  "fch1": 1510.25
  "filelist": [
    "./filfromfil2.fil"
  ]
  "filename": "./filfromfil2.fil"
  "foff": -1.75
  "format": "fil"
  "frequency_decimation_factor": 1
  "gb": -0.15836762417033684
  "gl": -169.27316727335108
  "native_foff": -1.75
  "native_nbits": 32
  "native_nchans": 1
  "native_nspectra": 15360.0
  "native_tsamp": 0.00014628571428571428
  "nbits": 32
  "nchans": 1
  "npol": 1
  "nspectra": 15360
  "poln_order": "I"
  "ra_deg": 272.4628333333333
  "source_name": "J1810-197_A2"
  "time_decimation_factor": 1
  "tsamp": 0.00014628571428571428
  "tstart": 59851.8768287037
  "tstart_utc": "2022-09-29T21:02:38.000"

In [55]:
writer_object = Writer(
    ...: Your(path_fil_original),
    ...: nstart=0,
    ...: nsamp=15360,
    ...: c_min=0,
    ...: c_max=31,
    ...: highest_frequency_first=True,
    ...: outdir ="./",
    ...: outname = "filfromfil3",
    ...: )
writer_object.to_fil()
Your("./filfromfil3.fil").your_header

Output()

Unified Header:
  "basename": "filfromfil3"
  "bw": -54.25
  "center_freq": 1482.25
  "dec_deg": -19.731091666666668
  "dtype": "float32"
  "fch1": 1508.5
  "filelist": [
    "./filfromfil3.fil"
  ]
  "filename": "./filfromfil3.fil"
  "foff": -1.75
  "format": "fil"
  "frequency_decimation_factor": 1
  "gb": -0.15836762417033684
  "gl": -169.27316727335108
  "native_foff": -1.75
  "native_nbits": 32
  "native_nchans": 31
  "native_nspectra": 15360.0
  "native_tsamp": 0.00014628571428571428
  "nbits": 32
  "nchans": 31
  "npol": 1
  "nspectra": 15360
  "poln_order": "I"
  "ra_deg": 272.4628333333333
  "source_name": "J1810-197_A2"
  "time_decimation_factor": 1
  "tsamp": 0.00014628571428571428
  "tstart": 59851.8768287037
  "tstart_utc": "2022-09-29T21:02:38.000"

In [56]:
writer_object = Writer(
    ...: Your(path_fil_original),
    ...: nstart=0,
    ...: nsamp=15360,
    ...: c_min=0,
    ...: c_max=31,
    ...: highest_frequency_first=False,
    ...: outdir ="./",
    ...: outname = "filfromfil4",
    ...: )
writer_object.to_fil()
Your("./filfromfil4.fil").your_header

Output()

Unified Header:
  "basename": "filfromfil4"
  "bw": 54.25
  "center_freq": 1482.25
  "dec_deg": -19.731091666666668
  "dtype": "float32"
  "fch1": 1456.0
  "filelist": [
    "./filfromfil4.fil"
  ]
  "filename": "./filfromfil4.fil"
  "foff": 1.75
  "format": "fil"
  "frequency_decimation_factor": 1
  "gb": -0.15836762417033684
  "gl": -169.27316727335108
  "native_foff": 1.75
  "native_nbits": 32
  "native_nchans": 31
  "native_nspectra": 15360.0
  "native_tsamp": 0.00014628571428571428
  "nbits": 32
  "nchans": 31
  "npol": 1
  "nspectra": 15360
  "poln_order": "I"
  "ra_deg": 272.4628333333333
  "source_name": "J1810-197_A2"
  "time_decimation_factor": 1
  "tsamp": 0.00014628571428571428
  "tstart": 59851.8768287037
  "tstart_utc": "2022-09-29T21:02:38.000"

In [57]:
writer_object = Writer(
    ...: Your(path_fil_original),
    ...: nstart=0,
    ...: nsamp=15360,
    ...: c_min=31,
    ...: c_max=0,
    ...: highest_frequency_first=False,
    ...: outdir ="./",
    ...: outname = "filfromfil5",
    ...: )
writer_object.to_fil()
Your("./filfromfil5.fil").your_header

Output()

Unified Header:
  "basename": "filfromfil5"
  "bw": 1.75
  "center_freq": 1510.25
  "dec_deg": -19.731091666666668
  "dtype": "float32"
  "fch1": 1510.25
  "filelist": [
    "./filfromfil5.fil"
  ]
  "filename": "./filfromfil5.fil"
  "foff": 1.75
  "format": "fil"
  "frequency_decimation_factor": 1
  "gb": -0.15836762417033684
  "gl": -169.27316727335108
  "native_foff": 1.75
  "native_nbits": 32
  "native_nchans": 1
  "native_nspectra": 15360.0
  "native_tsamp": 0.00014628571428571428
  "nbits": 32
  "nchans": 1
  "npol": 1
  "nspectra": 15360
  "poln_order": "I"
  "ra_deg": 272.4628333333333
  "source_name": "J1810-197_A2"
  "time_decimation_factor": 1
  "tsamp": 0.00014628571428571428
  "tstart": 59851.8768287037
  "tstart_utc": "2022-09-29T21:02:38.000"